In [1]:
import pandas as pd
import random
from sqlalchemy import create_engine

from snorkel.labeling import labeling_function

## Load Data

### DataFrame

In [5]:
# load the data by choosing a random 10% of the data
# is the data ordered? am I missing a large chunk due to my partial loading?
p = 0.01
responses = pd.read_csv('../data/moralmachine/SharedResponses.csv', skiprows=lambda i: i>0 and random.random() > p)

### SQL Engine

In [6]:
# create SQL DB with data
engine = create_engine("sqlite:///../data/moralmachine.db", echo=False)
responses.to_sql("sharedresponses", con=engine)

ValueError: Table 'sharedresponses' already exists.

In [ ]:
result = engine.execute("SELECT * FROM sharedresponses LIMIT 10")
result.fetchall()

## Data Exploration

In [ ]:
responses.describe()

In [ ]:
## For viewing the possible values for each feature
for feature in responses.columns:
    print(feature)
    print(responses[feature].unique())
    print(responses[feature].unique().shape)

In [ ]:
## What does a single user's session/response look like?
responses_random = responses[responses['ScenarioTypeStrict'] == 'Random']
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    ## see a random user's session
#     display(responses_random[responses_random['ExtendedSessionID'] == responses_random['ExtendedSessionID'].sample().values[0]])
    
    ## see the user with nth most responses
    n = 100
    display(responses_random[responses_random['ResponseID'] == responses_random.groupby(by='ResponseID').size().sort_values(ascending=False).index[n]])


In [ ]:
## group alternatives pairwise!
# take only responses where both alternatives are present in the random sample - for a full sample, this step isn't necessary
responses_grouped = responses.groupby(by='ResponseID').filter(lambda g: g.shape[0] < 2)
responses_grouped.shape

## Snorkel Labeling